In [13]:
# get our db url
# create db engine via sqlalchemy

In [14]:
from env import config
from sqlalchemy import create_engine, text, Column, Integer, String , BigInteger, Date, Float, Boolean
from sqlalchemy.orm import sessionmaker, DeclarativeBase

class Base(DeclarativeBase):
    pass

In [15]:
DATABASE_URL = config("FLIGHT_PRICES_DATABSE_URL", default=None)
DATABASE_URL is not None

True

In [16]:
engine = create_engine(str(DATABASE_URL))

In [17]:
# index
# flightDate             datetime64[ns]
# startingAirport                object
# destinationAirport             object
# isBasicEconomy                   bool
# isRefundable                     bool
# isNonStop                        bool
# segmentsAirlineName            object
# totalFare                       int64
# dtype: object

In [18]:
class FlightPrice(Base):
    __tablename__ = "flight_prices"
    __table_args__ = {"extend_existing": True}
    id = Column("index",BigInteger, primary_key=True)
    flightDate = Column("flightDate",Date)
    startingAirport  = Column(String)
    destinationAirport = Column(String)
    isBasicEconomy = Column(Boolean)
    isRefundable = Column(Boolean)
    isNonStop  = Column(Boolean)
    segmentsAirlineName  = Column(String)
    totalFare = Column(Integer)


In [19]:
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()
queryset = session.query(FlightPrice).limit(10).all()
for row_value in queryset:
    print(row_value.id, row_value.flightDate, row_value.totalFare)

session.close()

0 2022-04-17 00:00:00 273
1 2022-04-17 00:00:00 301
2 2022-04-17 00:00:00 352
3 2022-04-17 00:00:00 355
4 2022-04-17 00:00:00 307
5 2022-04-17 00:00:00 302
6 2022-04-17 00:00:00 307
7 2022-04-17 00:00:00 492
8 2022-04-17 00:00:00 543
9 2022-04-17 00:00:00 467


In [20]:
from pydantic import BaseModel, ConfigDict
from datetime import date
class FlightPriceSchema(BaseModel):
    id: int
    flightDate: date
    startingAirport: str
    destinationAirport: str
    isBasicEconomy: bool
    isRefundable: bool
    isNonStop: bool
    segmentsAirlineName: str
    totalFare: int

    model_config = ConfigDict(
        from_attributes=True
    )



In [21]:
SessionLocal = sessionmaker(bind=engine)
session = SessionLocal()
queryset = session.query(FlightPrice).limit(10).all()
for row_value in queryset:
       # Use model_validate instead of from_orm
        data = FlightPriceSchema.model_validate(row_value)
        # Use model_dump_json instead of json()
        print(data.model_dump_json(),'\n')

session.close()

{"id":0,"flightDate":"2022-04-17","startingAirport":"ATL","destinationAirport":"BOS","isBasicEconomy":false,"isRefundable":false,"isNonStop":false,"segmentsAirlineName":"American Airlines||American Airlines","totalFare":273} 

{"id":1,"flightDate":"2022-04-17","startingAirport":"ATL","destinationAirport":"BOS","isBasicEconomy":false,"isRefundable":false,"isNonStop":false,"segmentsAirlineName":"Spirit Airlines||Spirit Airlines","totalFare":301} 

{"id":2,"flightDate":"2022-04-17","startingAirport":"ATL","destinationAirport":"BOS","isBasicEconomy":false,"isRefundable":false,"isNonStop":false,"segmentsAirlineName":"United||United","totalFare":352} 

{"id":3,"flightDate":"2022-04-17","startingAirport":"ATL","destinationAirport":"BOS","isBasicEconomy":false,"isRefundable":false,"isNonStop":true,"segmentsAirlineName":"American Airlines","totalFare":355} 

{"id":4,"flightDate":"2022-04-17","startingAirport":"ATL","destinationAirport":"BOS","isBasicEconomy":false,"isRefundable":false,"isNonSto

In [22]:
engine.dispose()